In [1]:
import tomotopy as tp
from gensim import corpora
from gensim.models import CoherenceModel
from tqdm import tqdm

#plot
import matplotlib
matplotlib.use("TkAgg")  # Define o backend ANTES de importar pyplot

import matplotlib.pyplot as plt  # Só importar depois de definir o backend

from nltk import tokenize
import pandas as pd

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode


In [2]:
df = pd.read_csv('../datasets/fakeTelegram.BR_2022.csv')

In [3]:
len(df)

557586

In [4]:
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Wanda Silva,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Wanda Silva,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Wanda Silva,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Wanda Silva,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Wanda Silva,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637


In [5]:
ids_para_remover = ['b73c0b674b28d87375d78dcae2f1a21f','62b3aa83e4df122e48cb1f97583e403f',
                    '893874da7344daaa00b8b5b3dc295b59','38737f3c7a07586c4efbfa39aa345705',
                    'da4cbbc1b7b1192ad83bcdae613ac2c3', 'bf66a467459264e0d33e4c8518a82827',
                    '46e009035e6150d656d0c194db88fb07','811541f6ce49347d1798e8dd2d3cbf83',
                    '1d2a0a63333d12ad188e9e6f7f2e0419','7c379613d4da713fe959d8c7ddc11ce2',
                    'debe17fd10d504076d2df8682f63bfc6','e9713ae04a02a810d6f33dd956f42794',
                    'd6c77928db26721ce46aca2d549780f0','8e53464d3b01eea3e39d07e51ecbb1b4',
                    'bacd40da04dd7d13f646993bdcf8e79d','2273d1167a6212812d95dc8fadbae78e',
                    'cd3bb1bdf75be7595e6373171a5c2225','add8c1ba533c5e5450d92c061a5ee7bf',
                    'e198f90df1995528531dd43db0c935ea', '06dc9ac55ed64caab2bd97e9ab717302',
                    'c0110feb539d212836605b66192722dd','f71912700ac5331415408ce229681359',
                    '2ee692357a9c948351c43a9540e859ae', '25b66278176dabe814dfc25a405a2470',
                    '08b21cb1e7de74ef5fe1085230075523']

In [6]:
# Filtrar o DataFrame para remover as linhas com esses IDs
df_filtrado = df[~df['id_member_anonymous'].isin(ids_para_remover)]
df_filtrado

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Wanda Silva,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Wanda Silva,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Wanda Silva,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Wanda Silva,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Wanda Silva,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-...,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro ...",5,2022-11-16 14:49:39.146502,0.1027,NaN,575796,Wanda Silva,Url,telegram,NaN,25e43b6a58b848c43ad5b5f9e979822a
557582,2022-11-11 12:09:08,NaN,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,5,2022-11-16 14:49:39.847434,0.0000,NaN,1286443,Wanda Silva,Url,telegram,NaN,657949d03e4088f6b332e2686ccd3221
557583,2022-11-11 12:09:47,NaN,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",5,2022-11-16 14:49:39.922279,0.0000,NaN,13294,Wanda Silva,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e
557584,2022-11-11 12:09:46,NaN,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro.\n\n...",5,2022-11-16 14:49:39.992932,0.0000,NaN,1286444,Wanda Silva,Imagem,telegram,NaN,a21848a61045380a6483866daed0ca0e


In [7]:
# Removendo linhas com valores NaN na coluna 'text_content_anonymous'
df_filtrado = df_filtrado.dropna(subset=['text_content_anonymous'])

# Removendo trava_zap
df_filtrado = df_filtrado[df_filtrado['trava_zap'] == False]



#df_filtrado = df_filtrado[0:100]
len(df_filtrado)

443759

In [8]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

custom_stopwords = {
    't', 'be', 'nao', 'youtu', 'vai', 'av', 'ja', 'to', 'the', 'this', 'i', 'and', 
    'you', 'y', 'www', 'sao', 'pois', 'contra', 'user', 'ai', 'so', 'gente', 'voce', 'of', 
    'ola', 'gift', 'card', 'kwaivideo', 'r', 'bom', 'q', 'vc', 'vcs', 'pra','ta', 'phone', 'ok', 'la',
    'sera', 'ha', 'aqui', 'ate', 'dia', 'mc', 'im', 'tmj', 'pix', 'g', 'diz', 'ti', 'etc', 'tudo', 
    'todo', 'toda', 'youtube', 'g1', 'm', 'instagram', 'fb', 'in', 'link', 'was', 'blocked', 'kk'
}

stop_words.update(custom_stopwords)

def preprocess_text(text):

    # Função para extrair e substituir o domínio da URL
    def substituir_dominios(texto):
        # Função para extrair e substituir o domínio da URL
        def extrair_dominio(url):
            # Remove o protocolo (http://, https://, etc.) e o "www." se presente
            dominio = re.sub(r'^https?://(?:www\.)?|www\.', '', url)
            # Remove o caminho e parâmetros da URL
            dominio = re.split(r'[/?#]', dominio)[0]
            # Retorna a parte principal do domínio (antes do primeiro ponto)
            return dominio.split('.')[0]

        # Substitui URLs por seus domínios principais
        return re.sub(r'https?://(?:www\.)?\S+|www\.\S+', lambda match: extrair_dominio(match.group(0)), texto)

    # Substituir domínios
    text = substituir_dominios(text)

    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = unidecode(text)

    #Remover Pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs e menções
    #text = re.sub(r'http\S+|www\S+|https\S+|@\w+', '', text)

    # Substitui emojis repetidos por apenas um
    text = re.sub(r'([\U00010000-\U0010FFFF])\1+', r'\1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|[\U0001FA70-\U0001FAFF])\1+', r'\1', text)


    # Remove espaços em branco extras (início ou final) e múltiplos espaços no meio do texto
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove pontuações e caracteres especiais
    #text = re.sub(r'[^\w\s]', '', text)

    # Ajusta risadas "kkk" ou mais para "kk"
    text = re.sub(r'k{2,}|K{2,}', 'kk', text)

    # Ajusta risadas "haha" ou mais para "haha"
    text = re.sub(r'(ha){2,}', 'haha', text, flags=re.IGNORECASE)

    # Ajusta risadas "kaka" ou mais para "kaka"
    text = re.sub(r'(ka){2,}', 'kaka', text, flags=re.IGNORECASE)

    # Remove as stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])


    return text


[nltk_data] Downloading package punkt to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:

df_geral = df_filtrado.copy()

# Aplicar o pré-processamento à coluna de texto
df_geral['text_processed'] = df_geral['text_content_anonymous'].apply(preprocess_text)

df_geral.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5,text_processed
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,...,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Wanda Silva,Texto,telegram,NaN,NaN,entao fato renato audio ouvi whatsapp ocorreu ...
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",...,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Wanda Silva,Texto,telegram,NaN,NaN,saiu presidente 8 horas atras infelizmente con...
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",...,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Wanda Silva,Texto,telegram,NaN,NaN,parte quase feita segundo turno completamos pa...
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,...,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Wanda Silva,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb,achei seita maconarica
5,2022-10-05 06:28:30,NaN,b52442a5fbc459ae590dca0d215e32f9,NaN,NaN,NaN,False,False,False,Kķkkkkk to rindo até agora....Quem disse q ia ...,...,2022-10-05 06:29:29.046694,0.7003,0.197813,2735,Wanda Silva,Texto,telegram,NaN,NaN,rindo agoraquem disse ia fazer acordo diabo pr...


### Calculando a diversidade dos tópicos
A diversidade de tópicos mede quantas palavras únicas existem entre os top-N termos de todos os tópicos. 

In [10]:
def topic_diversity(model, top_n=10):
    topic_words = []
    for k in range(model.k):  
        words = [word for word, _ in model.get_topic_words(k, top_n=top_n)]  
        topic_words.extend(words)
    unique_words = set(topic_words)
    return len(unique_words) / (top_n * model.k)


### IRBO 

IRBO avalia quanto os tópicos são distintos entre si, usando o Rank-Biased Overlap (RBO) invertido. Quanto maior o IRBO, mais diferentes são os tópicos

In [11]:
import rbo

def compute_irbo_ptm(model, top_n=10):
    topics = []
    for k in range(model.k):
        topic_terms = [word for word, _ in model.get_topic_words(k, top_n=top_n)]
        topics.append(topic_terms)

    n = len(topics)
    total_irbo = 0
    count = 0

    for i in range(n):
        for j in range(i + 1, n):
            rbo_score = rbo.RankingSimilarity(topics[i], topics[j]).rbo()
            total_irbo += (1 - rbo_score)  # IRBO = 1 - RBO
            count += 1

    return total_irbo / count if count > 0 else 0


In [12]:
# Processamento
token_espaco = tokenize.WhitespaceTokenizer()
token_pontuacao = tokenize.WordPunctTokenizer()

In [13]:
def compute_coherence_ptm(texts, id2word, coherence, start=2, limit=30, step=2):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Treinando PTModel ({coherence})"):
        model = tp.PTModel(k=num_topics, seed=42)
        for doc in texts:
            model.add_doc(doc)
        model.train(100)
        model_list.append(model)

        # Coherence do gensim precisa de tópicos no formato: lista de listas de palavras
        topics = []
        for k in range(model.k):
            topic_words = [word for word, _ in model.get_topic_words(k, top_n=10)]
            topics.append(topic_words)

        coh_model = CoherenceModel(topics=topics, texts=texts, dictionary=id2word, coherence=coherence)
        coherence_score = coh_model.get_coherence()
        coherence_values.append(coherence_score)

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]

    return optimal_num_topics, model_list, coherence_values


In [14]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto]


## Modelando os Tópicos sem filtro 

In [15]:
texto_processado = df_geral['text_processed'].apply(processar_texto)

# Corpus e Dicionário (para CoherenceModel do gensim)
id2word = corpora.Dictionary(texto_processado)
corpus = [id2word.doc2bow(text) for text in texto_processado]
texts = texto_processado.tolist()

In [16]:
def calculate_ptm_coherence_for_metrics(texts, id2word, metrics, start=2, limit=30, step=2):
    results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_ptm(
            texts=texts,
            id2word=id2word,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

ptm_coherence_results = calculate_ptm_coherence_for_metrics(
    texts=texto_processado.tolist(),
    id2word=id2word,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)


Treinando PTModel (c_v):   0%|          | 0/14 [00:00<?, ?it/s]C:\Users\Melissa Felipe\AppData\Local\Temp\ipykernel_8628\2436467606.py:10: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
Treinando PTModel (c_npmi): 100%|██████████| 14/14 [1:05:52<00:00, 282.32s/it]


In [ ]:
# Mostrar resultados
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), ptm_coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {ptm_coherence_results[metric]['num_topics']}")



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.423
Qtd. tópicos = 4 -> Coerência = 0.4015
Qtd. tópicos = 6 -> Coerência = 0.5568
Qtd. tópicos = 8 -> Coerência = 0.4695
Qtd. tópicos = 10 -> Coerência = 0.5098
Qtd. tópicos = 12 -> Coerência = 0.5096
Qtd. tópicos = 14 -> Coerência = 0.5035
Qtd. tópicos = 16 -> Coerência = 0.5623
Qtd. tópicos = 18 -> Coerência = 0.5592
Qtd. tópicos = 20 -> Coerência = 0.515
Qtd. tópicos = 22 -> Coerência = 0.5486
Qtd. tópicos = 24 -> Coerência = 0.567
Qtd. tópicos = 26 -> Coerência = 0.5159
Qtd. tópicos = 28 -> Coerência = 0.5897
Melhor número de tópicos segundo C_V: 28

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -3.4423
Qtd. tópicos = 4 -> Coerência = -5.3086
Qtd. tópicos = 6 -> Coerência = -3.825
Qtd. tópicos = 8 -> Coerência = -6.2338
Qtd. tópicos = 10 -> Coerência = -4.6999
Qtd. tópicos = 12 -> Coerência = -4.8487
Qtd. tópicos = 14 -> Coerência = -5.0896
Qtd. tópicos = 16 -> Coerência = -4.6374
Qtd. tópic

In [18]:
# Escolha da métrica
chosen_metric = 'c_npmi'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_c_npmi.csv', index=False)
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"grupo, canal, abaixo, verdade, vpn, poder, 3, ...","[0.015988091006875038, 0.012962390668690205, 0..."
1,1,"13, intervencao, 22, militar, rua, pau, mao, b...","[0.12448844313621521, 0.05460965633392334, 0.0..."
2,2,"55, militar, batalhao, sp, rua, centro, comand...","[0.022184057161211967, 0.011170903220772743, 0..."
3,3,"telegram, ganhar, welcome, professional, group...","[0.02615315653383732, 0.017487330362200737, 0...."
4,4,"bolsonaro, brasil, todos, lula, presidente, es...","[0.012264122255146503, 0.00819872785359621, 0...."
5,5,"tse, superior, brazil, following, decision, co...","[0.024527493864297867, 0.022596294060349464, 0..."


In [19]:
div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")


Diversidade de Tópicos: 0.9
IRBO médio entre tópicos: 0.9806


In [20]:
# Escolha da métrica
chosen_metric = 'c_v'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_c_v.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7893
IRBO médio entre tópicos: 0.9788


,Tópico,Palavras,Pesos
0,0,"ruas, povo, armadas, forcas, quarteis, pode, p...","[0.01540481485426426, 0.01347713079303503, 0.0..."
1,1,"proibido, lula, bolsonaro, dizer, apoiadores, ...","[0.02882852964103222, 0.027155514806509018, 0...."
2,2,"reais, pl, 88888888, ptb, tmealexeconomia, tra...","[0.02544431947171688, 0.022112907841801643, 0...."
3,3,"ganhar, comecar, dinheiro, comigo, pode, bot, ...","[0.06236105412244797, 0.05637528747320175, 0.0..."
4,4,"55, brasil, grupos, 1, silva, souza, bolsonaro...","[0.18220603466033936, 0.007251132745295763, 0...."
5,5,"13, lula, presidente, vota, bolsonaro, rola, c...","[0.7298851013183594, 0.03948638588190079, 0.03..."
6,6,"km, br, cidade, local, turno, sul, nova, pesso...","[0.0082227298989892, 0.00807548500597477, 0.00..."
7,7,"milhoes, bilhoes, caso, escandalo, pt, preso, ...","[0.02420990727841854, 0.0157306008040905, 0.01..."
8,8,"intervencao, militar, pau, mao, federal, presi...","[0.31642836332321167, 0.1376071572303772, 0.10..."
9,9,"vpn, b, with, yn, bilan, men, erkaklari, sex, ...","[0.05719416216015816, 0.03432958573102951, 0.0..."


In [21]:
# Escolha da métrica
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_c_uci.csv', index=False)
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"grupo, canal, abaixo, verdade, vpn, poder, 3, ...","[0.015988091006875038, 0.012962390668690205, 0..."
1,1,"13, intervencao, 22, militar, rua, pau, mao, b...","[0.12448844313621521, 0.05460965633392334, 0.0..."
2,2,"55, militar, batalhao, sp, rua, centro, comand...","[0.022184057161211967, 0.011170903220772743, 0..."
3,3,"telegram, ganhar, welcome, professional, group...","[0.02615315653383732, 0.017487330362200737, 0...."
4,4,"bolsonaro, brasil, todos, lula, presidente, es...","[0.012264122255146503, 0.00819872785359621, 0...."
5,5,"tse, superior, brazil, following, decision, co...","[0.024527493864297867, 0.022596294060349464, 0..."


In [22]:
div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")


Diversidade de Tópicos: 0.9
IRBO médio entre tópicos: 0.9806


In [23]:
# Escolha da métrica
chosen_metric = 'u_mass'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_u_mass.csv', index=False)
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"13, lula, rua, militar, bolsonaro, sp, vpn, in...","[0.02766376920044422, 0.009874071925878525, 0...."
1,1,"bolsonaro, brasil, grupo, todos, presidente, t...","[0.008986602537333965, 0.006484880577772856, 0..."


In [24]:
div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")


Diversidade de Tópicos: 0.85
IRBO médio entre tópicos: 0.8575


## Modelagem de tópicos com filtro religioso

In [25]:
palavras_religiosas = [
    "deus", "jesus", "misericordia", "davi",
    "salomao", "reino", "templo", "conservador",
    "pentecostal", "rcc", "renovacao", "carismatic",
    "paulo ricardo", "bernardo kuster", "herege", "ateu",
    "jerico", "heresia"
]


In [26]:
# Função para verificar se uma palavra está relacionada à religião
def relacionada_religiao(word):
    word_lower = word.lower()
    palavras_religiosas_lower = [palavra.lower() for palavra in palavras_religiosas]

    # Verificando se alguma palavra da lista de palavras religiosas está presente
    return any(palavra in word_lower for palavra in palavras_religiosas_lower)


df_religiao = df_geral[df_geral['text_processed'].apply(lambda x: relacionada_religiao(x))]



In [27]:
texto_processado_religiao = df_religiao['text_processed'].apply(processar_texto)

In [28]:
# Corpus e Dicionário (para CoherenceModel do gensim)
id2word_religiao = corpora.Dictionary(texto_processado_religiao)
corpus_religiao = [id2word.doc2bow(text) for text in texto_processado_religiao]
texts_religiao = texto_processado_religiao.tolist()

In [29]:
def calculate_ptm_coherence_for_metrics(texts_religiao, id2word_religiao, metrics, start=2, limit=30, step=2):
    results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_ptm(
            texts=texts_religiao,
            id2word=id2word_religiao,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

ptm_coherence_results = calculate_ptm_coherence_for_metrics(
    texts_religiao=texto_processado_religiao.tolist(),
    id2word_religiao=id2word_religiao,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)


Treinando PTModel (c_v):   0%|          | 0/14 [00:00<?, ?it/s]C:\Users\Melissa Felipe\AppData\Local\Temp\ipykernel_8628\2436467606.py:10: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
Treinando PTModel (c_npmi): 100%|██████████| 14/14 [06:06<00:00, 26.20s/it]


In [ ]:

# Mostrar resultados
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), ptm_coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {ptm_coherence_results[metric]['num_topics']}")



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.3705
Qtd. tópicos = 4 -> Coerência = 0.4472
Qtd. tópicos = 6 -> Coerência = 0.4578
Qtd. tópicos = 8 -> Coerência = 0.4977
Qtd. tópicos = 10 -> Coerência = 0.5294
Qtd. tópicos = 12 -> Coerência = 0.474
Qtd. tópicos = 14 -> Coerência = 0.5626
Qtd. tópicos = 16 -> Coerência = 0.5306
Qtd. tópicos = 18 -> Coerência = 0.5804
Qtd. tópicos = 20 -> Coerência = 0.558
Qtd. tópicos = 22 -> Coerência = 0.5485
Qtd. tópicos = 24 -> Coerência = 0.5386
Qtd. tópicos = 26 -> Coerência = 0.5047
Qtd. tópicos = 28 -> Coerência = 0.5651
Melhor número de tópicos segundo C_V: 18

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -1.9174
Qtd. tópicos = 4 -> Coerência = -2.0417
Qtd. tópicos = 6 -> Coerência = -2.1842
Qtd. tópicos = 8 -> Coerência = -2.0041
Qtd. tópicos = 10 -> Coerência = -1.9605
Qtd. tópicos = 12 -> Coerência = -2.1358
Qtd. tópicos = 14 -> Coerência = -2.2907
Qtd. tópicos = 16 -> Coerência = -2.6599
Qtd. tóp

In [31]:
# Escolha da métrica
chosen_metric = 'c_npmi'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_religioso_c_npmi.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7286
IRBO médio entre tópicos: 0.921


,Tópico,Palavras,Pesos
0,0,"deus, todos, vamos, bolsonaro, brasil, preside...","[0.041431836783885956, 0.02248387597501278, 0...."
1,1,"reino, sobre, senhor, unido, 1, diante, eua, d...","[0.009836978279054165, 0.008336170576512814, 0..."
2,2,"grupo, liberdade, fortes, familiares, amigos, ...","[0.06820601969957352, 0.06814093887805939, 0.0..."
3,3,"brasil, fez, liberdade, 2, turno, povo, maior,...","[0.019763322547078133, 0.010239998809993267, 0..."
4,4,"pl, governador, senador, ptb, 22, 222, bolsona...","[0.029257144778966904, 0.014770093373954296, 0..."
5,5,"deus, porque, vida, jesus, ainda, povo, senhor...","[0.016343414783477783, 0.01142631471157074, 0...."
6,6,"bolsonaro, grupo, eleitores, 22, brasil, verda...","[0.0443139411509037, 0.040897972881793976, 0.0..."
7,7,"republica, stf, brasil, tribunal, militar, pod...","[0.013632675632834435, 0.010680408217012882, 0..."
8,8,"lula, brasil, pt, porque, entenderam, governo,...","[0.07856515049934387, 0.029402367770671844, 0...."
9,9,"eterno, dus, rei, bendito, nome, todos, paz, b...","[0.015613953582942486, 0.011178207583725452, 0..."


In [32]:
# Escolha da métrica
chosen_metric = 'c_v'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_religioso_c_v.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7389
IRBO médio entre tópicos: 0.9438


,Tópico,Palavras,Pesos
0,0,"deus, comigo, existe, repitam, porque, entende...","[0.06948002427816391, 0.05935677886009216, 0.0..."
1,1,"brasil, bolsonaro, anos, outro, poder, pt, sis...","[0.014896262437105179, 0.011328891851007938, 0..."
2,2,"senhor, jesus, nome, terra, pai, eterno, paz, ...","[0.021687233820557594, 0.012458330020308495, 0..."
3,3,"pl, ptb, bolsonaro, coronel, gov, sen, tenente...","[0.047786496579647064, 0.022010289132595062, 0..."
4,4,"brasil, fez, povo, br, coisas, 10, 2, deu, men...","[0.02202051691710949, 0.014850705862045288, 0...."
5,5,"lula, brasil, pt, favor, politica, sobre, quer...","[0.08825889229774475, 0.041373442858457565, 0...."
6,6,"grupo, liberdade, patria, familia, amigos, dir...","[0.07058853656053543, 0.06960321962833405, 0.0..."
7,7,"grupo, bolsonaro, eleitores, 22, brasil, verda...","[0.046749044209718704, 0.04629167169332504, 0...."
8,8,"direito, manifestacao, pode, caso, todos, inte...","[0.019073396921157837, 0.015212424099445343, 0..."
9,9,"presidente, bolsonaro, estao, deus, grupos, gr...","[0.020641479641199112, 0.02041035331785679, 0...."


In [33]:
# Escolha da métrica
chosen_metric = 'u_mass'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_religioso_u_mass.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.85
IRBO médio entre tópicos: 0.6125


,Tópico,Palavras,Pesos
0,0,"deus, todos, senhor, pode, bolsonaro, sobre, v...","[0.007684251293540001, 0.0060497550293803215, ..."
1,1,"deus, grupo, 55, bolsonaro, brasil, liberdade,...","[0.025675015524029732, 0.013673105277121067, 0..."


In [34]:
# Escolha da métrica
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_religioso_c_uci.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7286
IRBO médio entre tópicos: 0.921


,Tópico,Palavras,Pesos
0,0,"deus, todos, vamos, bolsonaro, brasil, preside...","[0.041431836783885956, 0.02248387597501278, 0...."
1,1,"reino, sobre, senhor, unido, 1, diante, eua, d...","[0.009836978279054165, 0.008336170576512814, 0..."
2,2,"grupo, liberdade, fortes, familiares, amigos, ...","[0.06820601969957352, 0.06814093887805939, 0.0..."
3,3,"brasil, fez, liberdade, 2, turno, povo, maior,...","[0.019763322547078133, 0.010239998809993267, 0..."
4,4,"pl, governador, senador, ptb, 22, 222, bolsona...","[0.029257144778966904, 0.014770093373954296, 0..."
5,5,"deus, porque, vida, jesus, ainda, povo, senhor...","[0.016343414783477783, 0.01142631471157074, 0...."
6,6,"bolsonaro, grupo, eleitores, 22, brasil, verda...","[0.0443139411509037, 0.040897972881793976, 0.0..."
7,7,"republica, stf, brasil, tribunal, militar, pod...","[0.013632675632834435, 0.010680408217012882, 0..."
8,8,"lula, brasil, pt, porque, entenderam, governo,...","[0.07856515049934387, 0.029402367770671844, 0...."
9,9,"eterno, dus, rei, bendito, nome, todos, paz, b...","[0.015613953582942486, 0.011178207583725452, 0..."


## Modelagem de tópicos com filtros religiosos e exclusão de político

In [35]:
palavras_politicas = [ "lula", "bolsonaro", "pt", "pl", "stf", "patria", "55", "22", "13", "senadores", "lulaladrao",
                       "urnas", "alexandre", "moraes", "comunismo", "eleicao", "eleicoes", "esquerda", "direita",
                         "presidente", "tse", "fraude", "voto", "turno", "ministro"]

In [36]:
def retirar_mensagens_com_palavras_politicas(word):
    word_lower = word.lower()
    palavras_politica_lower = [palavra.lower() for palavra in palavras_politicas]

    return any(palavra in word_lower for palavra in palavras_politica_lower)


df_politico = df_religiao[~df_religiao['text_processed'].apply(lambda x: retirar_mensagens_com_palavras_politicas(x))]



In [37]:
texto_processado_politico = df_politico['text_processed'].apply(processar_texto)

# Corpus e Dicionário (para CoherenceModel do gensim)
id2word_politico = corpora.Dictionary(texto_processado_politico)
corpus_politico = [id2word.doc2bow(text) for text in texto_processado_politico]
texts_politico = texto_processado_politico.tolist()

In [38]:
def calculate_ptm_coherence_for_metrics(texts_politico, id2word_politico, metrics, start=2, limit=30, step=2):
    results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_ptm(
            texts=texts_politico,
            id2word=id2word_politico,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

ptm_coherence_results = calculate_ptm_coherence_for_metrics(
    texts_politico=texto_processado_politico.tolist(),
    id2word_politico=id2word_politico,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)


Treinando PTModel (c_v):   0%|          | 0/14 [00:00<?, ?it/s]C:\Users\Melissa Felipe\AppData\Local\Temp\ipykernel_8628\2436467606.py:10: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(100)
Treinando PTModel (c_npmi): 100%|██████████| 14/14 [04:05<00:00, 17.51s/it]


In [ ]:
# Mostrar resultados
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), ptm_coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {ptm_coherence_results[metric]['num_topics']}")



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.4188
Qtd. tópicos = 4 -> Coerência = 0.5263
Qtd. tópicos = 6 -> Coerência = 0.4945
Qtd. tópicos = 8 -> Coerência = 0.4393
Qtd. tópicos = 10 -> Coerência = 0.4762
Qtd. tópicos = 12 -> Coerência = 0.4929
Qtd. tópicos = 14 -> Coerência = 0.5086
Qtd. tópicos = 16 -> Coerência = 0.4833
Qtd. tópicos = 18 -> Coerência = 0.5459
Qtd. tópicos = 20 -> Coerência = 0.553
Qtd. tópicos = 22 -> Coerência = 0.5263
Qtd. tópicos = 24 -> Coerência = 0.4587
Qtd. tópicos = 26 -> Coerência = 0.5217
Qtd. tópicos = 28 -> Coerência = 0.5633
Melhor número de tópicos segundo C_V: 28

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -3.6458
Qtd. tópicos = 4 -> Coerência = -5.1187
Qtd. tópicos = 6 -> Coerência = -3.2921
Qtd. tópicos = 8 -> Coerência = -6.0459
Qtd. tópicos = 10 -> Coerência = -5.2286
Qtd. tópicos = 12 -> Coerência = -5.7057
Qtd. tópicos = 14 -> Coerência = -5.1599
Qtd. tópicos = 16 -> Coerência = -4.9454
Qtd. tó

In [40]:
# Escolha da métrica
chosen_metric = 'c_npmi'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_politico_c_npmi.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.8333
IRBO médio entre tópicos: 0.784


,Tópico,Palavras,Pesos
0,0,"deus, obrigado, vida, nunca, hoje, porque, mim...","[0.045057445764541626, 0.01908472739160061, 0...."
1,1,"deus, povo, jesus, vamos, estao, brasil, agora...","[0.08858043700456619, 0.017645973712205887, 0...."
2,2,"deus, comigo, existe, repitam, pode, ganhar, d...","[0.16829875111579895, 0.1621929109096527, 0.15..."
3,3,"deus, senhor, todos, jesus, cristo, mundo, sob...","[0.02486061304807663, 0.024773763492703438, 0...."
4,4,"deus, grupo, brasil, acima, todos, canal, irma...","[0.01701209880411625, 0.016739599406719208, 0...."
5,5,"nada, reino, amor, amem, novo, unido, frente, ...","[0.01434880681335926, 0.012644341215491295, 0...."


In [41]:
# Escolha da métrica
chosen_metric = 'c_v'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_politico_c_v.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.7393
IRBO médio entre tópicos: 0.9435


,Tópico,Palavras,Pesos
0,0,"general, comando, militar, bsb, adeus, chefe, ...","[0.057559117674827576, 0.03683922067284584, 0...."
1,1,"el, en, pais, brasil, su, entrada, argentina, ...","[0.025346433743834496, 0.02112249843776226, 0...."
2,2,"jesus, biblia, nunca, deus, cristo, salva, esq...","[0.06531991064548492, 0.028874173760414124, 0...."
3,3,"deus, vamos, brasil, fe, todos, fazer, vitoria...","[0.0818667858839035, 0.06584196537733078, 0.03..."
4,4,"enquanto, pe, jejum, esperar, sp, dizia, envia...","[0.02156415767967701, 0.01319317426532507, 0.0..."
5,5,"feedback, pode, comigo, dinheiro, bot, ganhar,...","[0.041292306035757065, 0.03921627253293991, 0...."
6,6,"hoje, novo, vida, alcancar, objetivos, poder, ...","[0.059319380670785904, 0.048707686364650726, 0..."
7,7,"deus, todos, brasil, acima, pais, amor, estao,...","[0.10897503793239594, 0.03354981169104576, 0.0..."
8,8,"dias, deus, liberdade, familia, 15, data, ucra...","[0.050779588520526886, 0.02316383086144924, 0...."
9,9,"acima, dentro, c, s, coronel, daniel, poder, r...","[0.025962961837649345, 0.014162687584757805, 0..."


In [42]:
# Escolha da métrica
chosen_metric = 'u_mass'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_politico_u_mass.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.8333
IRBO médio entre tópicos: 0.784


,Tópico,Palavras,Pesos
0,0,"deus, obrigado, vida, nunca, hoje, porque, mim...","[0.045057445764541626, 0.01908472739160061, 0...."
1,1,"deus, povo, jesus, vamos, estao, brasil, agora...","[0.08858043700456619, 0.017645973712205887, 0...."
2,2,"deus, comigo, existe, repitam, pode, ganhar, d...","[0.16829875111579895, 0.1621929109096527, 0.15..."
3,3,"deus, senhor, todos, jesus, cristo, mundo, sob...","[0.02486061304807663, 0.024773763492703438, 0...."
4,4,"deus, grupo, brasil, acima, todos, canal, irma...","[0.01701209880411625, 0.016739599406719208, 0...."
5,5,"nada, reino, amor, amem, novo, unido, frente, ...","[0.01434880681335926, 0.012644341215491295, 0...."


In [43]:
# Escolha da métrica
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_uci'

# Obter o melhor modelo treinado
best_index = ptm_coherence_results[chosen_metric]['coherence_values'].index(
    max(ptm_coherence_results[chosen_metric]['coherence_values'])
)
best_model = ptm_coherence_results[chosen_metric]['model_list'][best_index]

# Gerar DataFrame com tópicos
def get_topics_dataframe(model, top_n=10):
    topics_data = []
    for topic_id in range(model.k):
        words_probs = model.get_topic_words(topic_id, top_n=top_n)
        topic_words = [word for word, prob in words_probs]
        topic_probs = [prob for word, prob in words_probs]
        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(topic_words),
            'Pesos': topic_probs
        })
    return pd.DataFrame(topics_data)

df_topicos = get_topics_dataframe(best_model)

df_topicos.to_csv('./resultados_pseudo/telegram/pseudo_telegram_politico_c_uci.csv', index=False)

div = topic_diversity(best_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo_score = compute_irbo_ptm(best_model)
print(f"IRBO médio entre tópicos: {round(irbo_score, 4)}")

df_topicos.head(10)

Diversidade de Tópicos: 0.8333
IRBO médio entre tópicos: 0.784


,Tópico,Palavras,Pesos
0,0,"deus, obrigado, vida, nunca, hoje, porque, mim...","[0.045057445764541626, 0.01908472739160061, 0...."
1,1,"deus, povo, jesus, vamos, estao, brasil, agora...","[0.08858043700456619, 0.017645973712205887, 0...."
2,2,"deus, comigo, existe, repitam, pode, ganhar, d...","[0.16829875111579895, 0.1621929109096527, 0.15..."
3,3,"deus, senhor, todos, jesus, cristo, mundo, sob...","[0.02486061304807663, 0.024773763492703438, 0...."
4,4,"deus, grupo, brasil, acima, todos, canal, irma...","[0.01701209880411625, 0.016739599406719208, 0...."
5,5,"nada, reino, amor, amem, novo, unido, frente, ...","[0.01434880681335926, 0.012644341215491295, 0...."
